In [5]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [6]:
def read_data(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-4]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    data = np.array(data).transpose(0, 2, 1)
    d_shape = data.shape
    return data.reshape(d_shape[0], 1, d_shape[1], d_shape[2])

def read_label(data_path):
    data = []
    # 1.txt
    file_names = os.listdir(data_path)
    file_names.sort(key=lambda x:int(x[:-10]))
    for file_name in file_names:
        file_path = os.path.join(data_path, file_name)
        signal_data = np.loadtxt(file_path)
        data.append(signal_data)
    return np.array(data)

In [7]:
train_data_path = "..//data//train_new//train_data"
train_label_path = "..//data//train_new//train_label"
test_data_path = "..//data//test//test_data"
test_label_path = "..//data//test//test_label"

train_data = read_data(train_data_path).transpose(0, 2, 3, 1) # 519
train_label = read_label(train_label_path)
test_data = read_data(test_data_path).transpose(0, 2, 3, 1) # 519
test_label = read_label(test_label_path)

print(train_data.shape)
print(train_label.shape)
print(test_data.shape)
print(test_label.shape)

# np.save("train_data.npy", train_data)
# np.save("train_label.npy", train_label)
# np.save("test_data.npy", test_data)
# np.save("test_label.npy", test_label)

(1022, 6, 1024, 1)
(1022, 1024)
(332, 6, 1024, 1)
(332, 1024)


In [8]:
# Training Parameters
learning_rate = 0.00001
num_steps = 150
batch_size = 32
display_step = 1
data_len = len(train_data)


# tf Graph input
X = tf.placeholder(tf.float32, [None, 6, 1024, 1])
Y = tf.placeholder(tf.float32, [None, 1024])

def cross_entropy(y_,output_map):
    return -tf.reduce_mean(y_*tf.log(tf.clip_by_value(output_map,1e-10,1.0))+(1-y_)*tf.log(tf.clip_by_value(1-output_map,1e-10,1.0)), name="cross_entropy")

# Create model
def conv_net(x):
    x = tf.pad(x, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv1_1 = tf.layers.conv2d(x, 64, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    conv1_1 = tf.pad(conv1_1, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv1_2 = tf.layers.conv2d(conv1_1, 64, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)

    
    conv2_1 = tf.layers.max_pooling2d(conv1_2, pool_size=[1, 2], strides=[1, 2], padding='valid')
    conv2_1 = tf.pad(conv2_1, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv2_2 = tf.layers.conv2d(conv2_1, 128, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    conv2_2 = tf.pad(conv2_2, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv2_3 = tf.layers.conv2d(conv2_2, 128, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    
    conv3_1 = tf.layers.max_pooling2d(conv2_3, pool_size=[1, 2], strides=[1, 2], padding='valid')
    
    conv3_1 = tf.pad(conv3_1, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv3_2 = tf.layers.conv2d(conv3_1, 256, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    conv3_2 = tf.pad(conv3_2, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv3_3 = tf.layers.conv2d(conv3_2, 256, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    conv3_3 = tf.pad(conv3_3, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv3_4 = tf.layers.conv2d(conv3_3, 256, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
   
    
    conv2_4_1 = tf.layers.conv2d_transpose(conv3_4, 128, kernel_size=[1, 2], strides=[1, 2], padding="VALID")
    conv2_4 = tf.concat([conv2_4_1, conv2_3], axis=3)
    conv2_4 = tf.pad(conv2_4, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv2_5 = tf.layers.conv2d(conv2_4, 128, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    conv2_5 = tf.pad(conv2_5, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv2_6 = tf.layers.conv2d(conv2_5, 128, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    
    conv1_3_1 = tf.layers.conv2d_transpose(conv2_6, 64, kernel_size=[1, 2], strides=[1, 2], padding="VALID")
    conv1_3 = tf.concat([conv1_2, conv1_3_1], axis=3)
    conv1_3 = tf.pad(conv1_3, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv1_4 = tf.layers.conv2d(conv1_3, 64, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    conv1_4 = tf.pad(conv1_4, paddings=[[0, 0], [0, 0], [7, 8], [0, 0]], mode="REFLECT")
    conv1_4 = tf.layers.conv2d(conv1_4, 64, kernel_size=[1, 16], padding='valid', activation=tf.nn.relu)
    conv1_5 = tf.layers.conv2d(conv1_4, 256, kernel_size=[6, 1], padding='valid', activation=tf.nn.relu)
    conv1_6 = tf.layers.conv2d(conv1_5, 1, kernel_size=[1, 1], padding='valid', activation=tf.nn.sigmoid)
    
    out = tf.reshape(conv1_6, [-1, 1024])
    return out

# Construct model
logits = conv_net(X)

# Define loss and optimizer
loss_op = cross_entropy(Y, logits)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
pred = tf.greater(logits, 0.5)
pred = tf.cast(pred, tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, Y), tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training


In [9]:
sess = tf.Session()

# Run the initializer
sess.run(init)

for step in range(1, num_steps+1):
    np.random.seed(step)
    indices = np.arange(train_data.shape[0])
    np.random.shuffle(indices)
    train_data_new = train_data[indices]
    train_label_new = train_label[indices]

    start_point = 0
    end_point = start_point + batch_size
    while(end_point < data_len):
        batch_x = train_data_new[start_point: end_point]
        batch_y = train_label_new[start_point: end_point]

        start_point += batch_size
        end_point += batch_size

        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
#                 print("Testing Accuracy:", \
#                         sess.run(accuracy, feed_dict={X: test_data[:128],
#                                                       Y: test_label[:128]}))
    print('***********************')
    print("Testing Accuracy:", \
                    sess.run(accuracy, feed_dict={X: test_data,
                                                  Y: test_label}))
    print('***********************')

print("Optimization Finished!")
sess.close()

Step 1, Minibatch Loss= 0.6961, Training Accuracy= 0.512
Step 1, Minibatch Loss= 0.7029, Training Accuracy= 0.390
Step 1, Minibatch Loss= 0.6968, Training Accuracy= 0.418
Step 1, Minibatch Loss= 0.6931, Training Accuracy= 0.506
Step 1, Minibatch Loss= 0.7010, Training Accuracy= 0.372
Step 1, Minibatch Loss= 0.6928, Training Accuracy= 0.482
Step 1, Minibatch Loss= 0.6880, Training Accuracy= 0.601
Step 1, Minibatch Loss= 0.6926, Training Accuracy= 0.527
Step 1, Minibatch Loss= 0.6909, Training Accuracy= 0.532
Step 1, Minibatch Loss= 0.6849, Training Accuracy= 0.636
Step 1, Minibatch Loss= 0.6931, Training Accuracy= 0.539
Step 1, Minibatch Loss= 0.6886, Training Accuracy= 0.560
Step 1, Minibatch Loss= 0.6874, Training Accuracy= 0.580
Step 1, Minibatch Loss= 0.6834, Training Accuracy= 0.639
Step 1, Minibatch Loss= 0.6829, Training Accuracy= 0.619
Step 1, Minibatch Loss= 0.6889, Training Accuracy= 0.568
Step 1, Minibatch Loss= 0.6891, Training Accuracy= 0.548
Step 1, Minibatch Loss= 0.6836,

Step 5, Minibatch Loss= 0.3581, Training Accuracy= 0.874
Step 5, Minibatch Loss= 0.3735, Training Accuracy= 0.858
Step 5, Minibatch Loss= 0.4007, Training Accuracy= 0.840
Step 5, Minibatch Loss= 0.4112, Training Accuracy= 0.879
Step 5, Minibatch Loss= 0.4756, Training Accuracy= 0.820
Step 5, Minibatch Loss= 0.5708, Training Accuracy= 0.762
Step 5, Minibatch Loss= 0.4763, Training Accuracy= 0.771
Step 5, Minibatch Loss= 0.4611, Training Accuracy= 0.820
Step 5, Minibatch Loss= 0.4983, Training Accuracy= 0.786
Step 5, Minibatch Loss= 0.4468, Training Accuracy= 0.812
Step 5, Minibatch Loss= 0.4414, Training Accuracy= 0.845
Step 5, Minibatch Loss= 0.3559, Training Accuracy= 0.876
Step 5, Minibatch Loss= 0.4790, Training Accuracy= 0.777
Step 5, Minibatch Loss= 0.4380, Training Accuracy= 0.805
Step 5, Minibatch Loss= 0.3726, Training Accuracy= 0.855
Step 5, Minibatch Loss= 0.6016, Training Accuracy= 0.734
***********************
Testing Accuracy: 0.7960073
***********************
Step 6, Mini

Step 9, Minibatch Loss= 0.2841, Training Accuracy= 0.898
***********************
Testing Accuracy: 0.83717585
***********************
Step 10, Minibatch Loss= 0.3319, Training Accuracy= 0.882
Step 10, Minibatch Loss= 0.3183, Training Accuracy= 0.871
Step 10, Minibatch Loss= 0.2190, Training Accuracy= 0.915
Step 10, Minibatch Loss= 0.2731, Training Accuracy= 0.898
Step 10, Minibatch Loss= 0.1866, Training Accuracy= 0.922
Step 10, Minibatch Loss= 0.5117, Training Accuracy= 0.782
Step 10, Minibatch Loss= 0.2330, Training Accuracy= 0.910
Step 10, Minibatch Loss= 0.3093, Training Accuracy= 0.898
Step 10, Minibatch Loss= 0.2987, Training Accuracy= 0.876
Step 10, Minibatch Loss= 0.3652, Training Accuracy= 0.845
Step 10, Minibatch Loss= 0.4406, Training Accuracy= 0.784
Step 10, Minibatch Loss= 0.4876, Training Accuracy= 0.779
Step 10, Minibatch Loss= 0.3692, Training Accuracy= 0.833
Step 10, Minibatch Loss= 0.3672, Training Accuracy= 0.811
Step 10, Minibatch Loss= 0.4933, Training Accuracy= 0.

Step 14, Minibatch Loss= 0.2748, Training Accuracy= 0.887
Step 14, Minibatch Loss= 0.3230, Training Accuracy= 0.879
Step 14, Minibatch Loss= 0.3248, Training Accuracy= 0.861
Step 14, Minibatch Loss= 0.2777, Training Accuracy= 0.887
Step 14, Minibatch Loss= 0.2695, Training Accuracy= 0.897
Step 14, Minibatch Loss= 0.3063, Training Accuracy= 0.871
Step 14, Minibatch Loss= 0.2847, Training Accuracy= 0.882
Step 14, Minibatch Loss= 0.2752, Training Accuracy= 0.875
Step 14, Minibatch Loss= 0.3478, Training Accuracy= 0.864
Step 14, Minibatch Loss= 0.3630, Training Accuracy= 0.843
Step 14, Minibatch Loss= 0.4051, Training Accuracy= 0.831
Step 14, Minibatch Loss= 0.2187, Training Accuracy= 0.918
Step 14, Minibatch Loss= 0.3229, Training Accuracy= 0.872
Step 14, Minibatch Loss= 0.3439, Training Accuracy= 0.845
Step 14, Minibatch Loss= 0.4568, Training Accuracy= 0.816
Step 14, Minibatch Loss= 0.3525, Training Accuracy= 0.854
Step 14, Minibatch Loss= 0.2609, Training Accuracy= 0.892
Step 14, Minib

Step 18, Minibatch Loss= 0.2804, Training Accuracy= 0.881
Step 18, Minibatch Loss= 0.3418, Training Accuracy= 0.864
Step 18, Minibatch Loss= 0.2986, Training Accuracy= 0.881
Step 18, Minibatch Loss= 0.3093, Training Accuracy= 0.872
Step 18, Minibatch Loss= 0.3002, Training Accuracy= 0.881
Step 18, Minibatch Loss= 0.2435, Training Accuracy= 0.893
Step 18, Minibatch Loss= 0.2989, Training Accuracy= 0.866
Step 18, Minibatch Loss= 0.3529, Training Accuracy= 0.826
Step 18, Minibatch Loss= 0.2645, Training Accuracy= 0.892
***********************
Testing Accuracy: 0.8453825
***********************
Step 19, Minibatch Loss= 0.2589, Training Accuracy= 0.892
Step 19, Minibatch Loss= 0.3562, Training Accuracy= 0.876
Step 19, Minibatch Loss= 0.3534, Training Accuracy= 0.845
Step 19, Minibatch Loss= 0.2366, Training Accuracy= 0.891
Step 19, Minibatch Loss= 0.3254, Training Accuracy= 0.838
Step 19, Minibatch Loss= 0.2444, Training Accuracy= 0.913
Step 19, Minibatch Loss= 0.3188, Training Accuracy= 0.

...

Step 132, Minibatch Loss= 0.2517, Training Accuracy= 0.886
Step 132, Minibatch Loss= 0.2034, Training Accuracy= 0.923
Step 132, Minibatch Loss= 0.2187, Training Accuracy= 0.903
Step 132, Minibatch Loss= 0.1681, Training Accuracy= 0.935
Step 132, Minibatch Loss= 0.2398, Training Accuracy= 0.878
Step 132, Minibatch Loss= 0.1950, Training Accuracy= 0.913
Step 132, Minibatch Loss= 0.2184, Training Accuracy= 0.905
Step 132, Minibatch Loss= 0.2675, Training Accuracy= 0.884
Step 132, Minibatch Loss= 0.2067, Training Accuracy= 0.902
Step 132, Minibatch Loss= 0.2900, Training Accuracy= 0.877
Step 132, Minibatch Loss= 0.1727, Training Accuracy= 0.927
Step 132, Minibatch Loss= 0.2612, Training Accuracy= 0.870
Step 132, Minibatch Loss= 0.2064, Training Accuracy= 0.911
Step 132, Minibatch Loss= 0.2344, Training Accuracy= 0.913
Step 132, Minibatch Loss= 0.1965, Training Accuracy= 0.923
Step 132, Minibatch Loss= 0.2508, Training Accuracy= 0.895
Step 132, Minibatch Loss= 0.2292, Training Accuracy= 0.9

Step 136, Minibatch Loss= 0.1841, Training Accuracy= 0.923
Step 136, Minibatch Loss= 0.2059, Training Accuracy= 0.905
Step 136, Minibatch Loss= 0.1579, Training Accuracy= 0.942
Step 136, Minibatch Loss= 0.2338, Training Accuracy= 0.899
Step 136, Minibatch Loss= 0.2698, Training Accuracy= 0.873
Step 136, Minibatch Loss= 0.3329, Training Accuracy= 0.860
Step 136, Minibatch Loss= 0.1912, Training Accuracy= 0.925
Step 136, Minibatch Loss= 0.3459, Training Accuracy= 0.854
Step 136, Minibatch Loss= 0.2555, Training Accuracy= 0.879
Step 136, Minibatch Loss= 0.2347, Training Accuracy= 0.907
Step 136, Minibatch Loss= 0.3073, Training Accuracy= 0.877
Step 136, Minibatch Loss= 0.2546, Training Accuracy= 0.875
Step 136, Minibatch Loss= 0.1771, Training Accuracy= 0.934
Step 136, Minibatch Loss= 0.1863, Training Accuracy= 0.932
Step 136, Minibatch Loss= 0.2406, Training Accuracy= 0.885
Step 136, Minibatch Loss= 0.1928, Training Accuracy= 0.915
Step 136, Minibatch Loss= 0.1395, Training Accuracy= 0.9

Step 140, Minibatch Loss= 0.2049, Training Accuracy= 0.912
Step 140, Minibatch Loss= 0.2743, Training Accuracy= 0.894
Step 140, Minibatch Loss= 0.1859, Training Accuracy= 0.929
Step 140, Minibatch Loss= 0.2320, Training Accuracy= 0.911
Step 140, Minibatch Loss= 0.2579, Training Accuracy= 0.892
Step 140, Minibatch Loss= 0.2107, Training Accuracy= 0.907
Step 140, Minibatch Loss= 0.2538, Training Accuracy= 0.887
Step 140, Minibatch Loss= 0.2179, Training Accuracy= 0.915
Step 140, Minibatch Loss= 0.1638, Training Accuracy= 0.924
***********************
Testing Accuracy: 0.8561776
***********************
Step 141, Minibatch Loss= 0.2527, Training Accuracy= 0.908
Step 141, Minibatch Loss= 0.1741, Training Accuracy= 0.931
Step 141, Minibatch Loss= 0.2919, Training Accuracy= 0.880
Step 141, Minibatch Loss= 0.2715, Training Accuracy= 0.857
Step 141, Minibatch Loss= 0.2395, Training Accuracy= 0.893
Step 141, Minibatch Loss= 0.2224, Training Accuracy= 0.919
Step 141, Minibatch Loss= 0.2097, Train

Step 145, Minibatch Loss= 0.2074, Training Accuracy= 0.924
Step 145, Minibatch Loss= 0.2089, Training Accuracy= 0.904
Step 145, Minibatch Loss= 0.1888, Training Accuracy= 0.925
Step 145, Minibatch Loss= 0.2285, Training Accuracy= 0.901
Step 145, Minibatch Loss= 0.2498, Training Accuracy= 0.917
Step 145, Minibatch Loss= 0.1708, Training Accuracy= 0.926
Step 145, Minibatch Loss= 0.2626, Training Accuracy= 0.878
Step 145, Minibatch Loss= 0.1933, Training Accuracy= 0.926
Step 145, Minibatch Loss= 0.2704, Training Accuracy= 0.882
Step 145, Minibatch Loss= 0.2658, Training Accuracy= 0.874
Step 145, Minibatch Loss= 0.2237, Training Accuracy= 0.905
Step 145, Minibatch Loss= 0.1987, Training Accuracy= 0.932
Step 145, Minibatch Loss= 0.2168, Training Accuracy= 0.903
Step 145, Minibatch Loss= 0.2467, Training Accuracy= 0.900
Step 145, Minibatch Loss= 0.2428, Training Accuracy= 0.886
Step 145, Minibatch Loss= 0.2904, Training Accuracy= 0.867
Step 145, Minibatch Loss= 0.1774, Training Accuracy= 0.9

Step 149, Minibatch Loss= 0.2255, Training Accuracy= 0.904
Step 149, Minibatch Loss= 0.2115, Training Accuracy= 0.893
Step 149, Minibatch Loss= 0.1431, Training Accuracy= 0.944
Step 149, Minibatch Loss= 0.1858, Training Accuracy= 0.927
Step 149, Minibatch Loss= 0.2100, Training Accuracy= 0.902
Step 149, Minibatch Loss= 0.1895, Training Accuracy= 0.932
Step 149, Minibatch Loss= 0.2391, Training Accuracy= 0.897
Step 149, Minibatch Loss= 0.1470, Training Accuracy= 0.942
Step 149, Minibatch Loss= 0.2217, Training Accuracy= 0.900
Step 149, Minibatch Loss= 0.2204, Training Accuracy= 0.899
Step 149, Minibatch Loss= 0.2198, Training Accuracy= 0.908
Step 149, Minibatch Loss= 0.1763, Training Accuracy= 0.929
Step 149, Minibatch Loss= 0.2246, Training Accuracy= 0.917
Step 149, Minibatch Loss= 0.2511, Training Accuracy= 0.904
Step 149, Minibatch Loss= 0.2595, Training Accuracy= 0.886
Step 149, Minibatch Loss= 0.2115, Training Accuracy= 0.908
Step 149, Minibatch Loss= 0.1814, Training Accuracy= 0.9

In [10]:
tf.__version__

'1.8.0'

In [ ]:
sess.close()